---
# Teste de Estratégias do Medium - Estratégia 01 - Larry Connors R3 Strategy

---

O objetivo desse projeto é analisar, pessoalmente, a viabilidade de uso de estratégias encontradas na plataforma Medium. 

Neste projeto em específico, será testada a estratégia de Larry Connors, especificada no canal Quantified Strategies, disponível em: <>

## 1. Obtenção e Preparação dos Dados e do Notebook

Serão utilizados os dados da B3, importados pelo MetaTrader, para fazer os testes. 

### 1.1. Importando bibliotecas

In [1]:
from backtesting import Cruzamentos
from conexao import importar_dados_historicos
import vectorbt as vbt 


import pandas as pd 
import numpy as np 
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import pytz 

import MetaTrader5 as mt5
import ta
import plotly.graph_objects as go
import mplfinance as mpf


### 1.2. Obtendo os dados

In [3]:

mt5.initialize()

timezone = pytz.timezone("America/Sao_Paulo")  


tickers = ['WDO$', 'WIN$', 'PETR4', 'VALE3', 'ITUB4', 'BBDC4', 'ABEV3', 'BBAS3', 'WEGE3', 'ITSA4']


for ticker in tickers:
    selected = mt5.symbol_select(ticker, True)
    if not selected:
        print(ticker, "não localizado")


start = datetime(2022, 1, 1)
end = datetime.today()
timeframe = mt5.TIMEFRAME_D1
dados_gerais = dict()


for ticker in tickers:
    dados_brutos = importar_dados_historicos(ticker, start, end, timeframe)
    dados_gerais[ticker] = dados_brutos 

dados = dados_gerais[tickers[0]].copy()

### 1.3. Parâmetros e Funções

Serão utilizados alguns parâmetros gerais, como curso de daytrade e de swingtrade (taxa de corretagem), bem como algumas funções para facilitar a análise dos resultados.

In [7]:
# Definindo parâmetros básicos
taxa_swing_trade = 0.030589/100
taxa_day_trade = 0.02 

In [8]:
def gerar_relatorio(conjunto_dados, freq, estrategia, fees=taxa_swing_trade):
    
    resultados = {'Ativo':[],
                  'RetornoTotal': [],
                  'Retorno_x_benchmark': [],
                  'MaxDrawdown': [],
                  'MaxDrawdownDuration': [],
                  'TotalTrades': [],
                  'WinRate': [],
                  'AvgWinningTrade':[],
                  'AvgLosingTrade':[],
                  'AvgLosingTradeDuration':[]}
    
    for ativo, dados_ohlc in conjunto_dados.items():
        portfolio = estrategia(dados_ohlc, freq=freq, fees=fees)
        resultados['Ativo'].append(ativo)
        resultados['RetornoTotal'].append(portfolio.stats()['Total Return [%]'])
        resultados['Retorno_x_benchmark'].append(portfolio.stats()['Total Return [%]'] - portfolio.stats()['Benchmark Return [%]'])
        resultados['MaxDrawdown'].append(portfolio.stats()['Max Drawdown [%]'])
        resultados['MaxDrawdownDuration'].append(portfolio.stats()['Max Drawdown Duration'])
        resultados['TotalTrades'].append(portfolio.stats()['Total Trades'])
        resultados['WinRate'].append(portfolio.stats()['Win Rate [%]'])
        resultados['AvgWinningTrade'].append(portfolio.stats()['Avg Winning Trade [%]'])
        resultados['AvgLosingTrade'].append(portfolio.stats()['Avg Losing Trade [%]'])
        resultados['AvgLosingTradeDuration'].append(portfolio.stats()['Avg Losing Trade Duration'])

    
    return pd.DataFrame(resultados)

## 2. Estratégia de Trading

Conforme site, as regras de trading (tradução própria) são: 

1. Valor de fechamento acima da média móvel de 200 dias 
2. IFR de 2 dias em queda por 3 dias seguidos e o primeiro IFR da sequência está abaixo de 60
3. O IFR de 2 dias atual deve está abaixo de 10 
4. Se todas as condições acima forem cumpridas, entrar no fechamento do dia 
5. Encerramento de posição quando o IFR de 2 dias ficar acima de 70.


Para essa estratégia, algumas dúvidas e/ou decisões que percebi foram:
- Não há como saber o IFR de 2 dias atual do preço de fechamento e realizar aquisição no mesmo dia, pois é necessário saber o valor de encerramento. 

Para isso, optou-se por seguir uma das duas soluções abaixo:
1. Abrir ou encerrar a posição no dia seguinte (depois de validar o dia anterior); ou 
2. Utilizar o IFR de preço de abertura




In [11]:
# Testando a solução 1 - abrindo ou encerrando posição no dia seguinte

def testar_estrategia(dados, freq, fees=taxa_swing_trade):
   
    # Obtendo IFR 
    ifr_2p = ta.momentum.RSIIndicator(close=dados.close, window=2).rsi()

    # Obtendo 3 dias de queda
    condicao_ifr_queda = ifr_2p.shift(3) < 60    # Primeiro candle deve ter ifr < 60
    for i in range(3):
        condicao_seguinte = ifr_2p.shift(i) < ifr_2p.shift(i+1)
        condicao_ifr_queda = condicao_ifr_queda & condicao_seguinte

    # Último IFR deve estar abaixo de 10
    condicao_ultimo_ifr = ifr_2p < 10

    # Sinal de entrada (long)
    sinal_entrada_long = (condicao_ifr_queda & condicao_ultimo_ifr).shift(1).fillna(False)    # Entrando no dia seguinte

    # Sinal de encerramento (long)
    sinal_encerramento_long = (ifr_2p > 70).shift(1).fillna(False)    # Encerrando no dia seguinte
    

    portfolio = vbt.Portfolio.from_signals(
        dados.close,
        entries=sinal_entrada_long,
        exits=sinal_encerramento_long,
       # short_entries=sinal_entrada_short, 
       # short_exits=sinal_saida_short,
        freq=freq,
        fees=fees
    )
    return portfolio 

resultado = gerar_relatorio(dados_gerais, '1d', testar_estrategia, fees=taxa_swing_trade)
resultado

,Ativo,RetornoTotal,Retorno_x_benchmark,MaxDrawdown,MaxDrawdownDuration,TotalTrades,WinRate,AvgWinningTrade,AvgLosingTrade,AvgLosingTradeDuration
0,WDO$,1.952956,19.947172,14.721325,373 days,17,58.823529,1.917234,-2.380982,10 days 03:25:42.857142857
1,WIN$,-10.509457,-10.495708,17.449284,584 days,15,66.666667,1.381944,-4.770073,11 days 00:00:00
2,PETR4,3.137573,-221.163866,16.500426,542 days,13,46.153846,2.762176,-1.860043,5 days 06:51:25.714285714
3,VALE3,38.449342,45.100830,8.743112,150 days,22,68.181818,3.208739,-2.011455,6 days 10:17:08.571428571
4,ITUB4,14.045091,-82.597091,7.500125,118 days,17,58.823529,2.379594,-1.441944,5 days 17:08:34.285714285
5,BBDC4,13.144687,25.167958,10.216757,163 days,16,75.000000,2.136539,-3.179852,6 days 18:00:00
6,ABEV3,5.796906,13.549396,9.448834,266 days,21,57.142857,1.725447,-1.629092,6 days 05:20:00
7,BBAS3,4.040800,-124.182197,14.013948,546 days,10,80.000000,1.801727,-4.953493,8 days 00:00:00
8,WEGE3,30.912530,-45.756318,12.257591,212 days,19,68.421053,3.215299,-2.211753,5 days 08:00:00
9,ITSA4,3.548562,-65.833036,10.380871,251 days,15,53.333333,1.694465,-1.396132,6 days 03:25:42.857142857


Pelos dados retornados, percebemos que há uma duração de drawdown muito grande, por vezes durando mais que um ano. 
Além disso, as perdas foram muito elevadas. 

Diante disso, buscarei alterar as regras para inclusão de um stoploss baseado no tempo, fazendo com que se encerre o trade após no máximo 90 dias. 

In [12]:
# Obtendo IFR 
ifr_2p = ta.momentum.RSIIndicator(close=dados.close, window=2).rsi()

# Obtendo 3 dias de queda
condicao_ifr_queda = ifr_2p.shift(3) < 60    # Primeiro candle deve ter ifr < 60
for i in range(3):
    condicao_seguinte = ifr_2p.shift(i) < ifr_2p.shift(i+1)
    condicao_ifr_queda = condicao_ifr_queda & condicao_seguinte

# Último IFR deve estar abaixo de 10
condicao_ultimo_ifr = ifr_2p < 10

# Sinal de entrada (long)
sinal_entrada_long = (condicao_ifr_queda & condicao_ultimo_ifr).shift(1).fillna(False)    # Entrando no dia seguinte

# Sinal de encerramento de long tradicional
sinal_encerramento_long = (ifr_2p > 70).shift(1).fillna(False)    # Encerrando no dia seguinte

# Adicionando stoploss de tempo
trade_duration = sinal_entrada_long.cumsum()  # Track how many trades have been opened
time_based_exit = (trade_duration - trade_duration.where(sinal_entrada_long).ffill()) >= 100


In [30]:
entrada_saida = pd.concat([sinal_entrada_long, sinal_encerramento_long], axis=1)
entrada_saida.columns = ['sianl_entrada_long', 'sinal_encerramento_long']
entrada_saida

,sianl_entrada_long,sinal_encerramento_long
time,,
2022-01-03,False,False
2022-01-04,False,False
2022-01-05,False,False
2022-01-06,False,False
2022-01-07,False,False
...,...,...
2024-09-02,False,True
2024-09-03,False,True
2024-09-04,False,True


time
2022-01-03    False
2022-01-04    False
2022-01-05    False
2022-01-06    False
2022-01-07    False
              ...  
2024-09-02     True
2024-09-03     True
2024-09-04     True
2024-09-05     True
2024-09-06    False
Name: rsi, Length: 652, dtype: bool

In [ ]:
# Testando a solução 1 - abrindo ou encerrando posição no dia seguinte

def testar_estrategia(dados, freq, fees=taxa_swing_trade):
   
    # Obtendo IFR 
    ifr_2p = ta.momentum.RSIIndicator(close=dados.close, window=2).rsi()

    # Obtendo 3 dias de queda
    condicao_ifr_queda = ifr_2p.shift(3) < 60    # Primeiro candle deve ter ifr < 60
    for i in range(3):
        condicao_seguinte = ifr_2p.shift(i) < ifr_2p.shift(i+1)
        condicao_ifr_queda = condicao_ifr_queda & condicao_seguinte

    # Último IFR deve estar abaixo de 10
    condicao_ultimo_ifr = ifr_2p < 10

    # Sinal de entrada (long)
    sinal_entrada_long = (condicao_ifr_queda & condicao_ultimo_ifr).shift(1).fillna(False)    # Entrando no dia seguinte

    # Sinal de encerramento de long tradicional
    sinal_encerramento_long = (ifr_2p > 70).shift(1).fillna(False)    # Encerrando no dia seguinte
    
    # Adicionando stoploss de tempo
    trade_duration = sinal_entrada_long.cumsum()  # Track how many trades have been opened
    time_based_exit = (trade_duration - trade_duration.where(sinal_entrada_long).ffill()) >= 100

    portfolio = vbt.Portfolio.from_signals(
        dados.close,
        entries=sinal_entrada_long,
        exits=sinal_encerramento_long,
       # short_entries=sinal_entrada_short, 
       # short_exits=sinal_saida_short,
        freq=freq,
        fees=fees
    )
    return portfolio 

resultado = gerar_relatorio(dados_gerais, '1d', testar_estrategia, fees=taxa_swing_trade)
resultado